In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import mean_squared_error, r2_score

plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)


# 재무정보 데이터 로드
financial_data = pd.read_excel('./financial_train.xlsx',index_col='종목코드') # 재무정보


# 비재무정보 데이터 로드
non_financial_data = pd.read_excel('./non_financial_train.xlsx',index_col='종목코드') # 비재무정보


In [59]:
financial_data.head()

,총자산,총자본회전율,부채비율,매출액증가율,자기자본회전율,당기순이익,레이블
종목코드,,,,,,,
360,6130.333968,0.846342,-32.596321,-0.036030,-2674.128824,-661.336250,1
420,719.682766,0.541509,-5.124169,-0.504883,-223.327431,-257.465802,1
790,2062.599327,1.009479,4.145668,0.311749,519.444543,-994.835473,1
800,18007.307005,0.978722,2.499255,0.751287,342.479670,131.586231,1
895,2098.958240,0.583371,1.620366,0.022887,152.864625,49.455226,1


In [60]:
financial_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900 entries, 360 to 900100
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총자산      900 non-null    float64
 1   총자본회전율   900 non-null    float64
 2   부채비율     900 non-null    float64
 3   매출액증가율   900 non-null    float64
 4   자기자본회전율  900 non-null    float64
 5   당기순이익    900 non-null    float64
 6   레이블      900 non-null    int64  
dtypes: float64(6), int64(1)
memory usage: 56.2 KB


In [61]:
non_financial_data.head()

,최대주주변경,대표이사변경,전환사채,파산신청,거래정지,불성실공시법인,레이블
종목코드,,,,,,,
360,0,0,0,0,0,0,1
420,0,0,0,0,2,0,1
790,0,0,0,0,1,0,1
800,0,0,2,0,1,0,1
895,0,0,1,0,0,0,1


In [62]:
non_financial_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900 entries, 360 to 900100
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   최대주주변경   900 non-null    int64
 1   대표이사변경   900 non-null    int64
 2   전환사채     900 non-null    int64
 3   파산신청     900 non-null    int64
 4   거래정지     900 non-null    int64
 5   불성실공시법인  900 non-null    int64
 6   레이블      900 non-null    int64
dtypes: int64(7)
memory usage: 56.2 KB


# **재무 / 비재무 데이터 비교**

In [64]:
# 재무정보에서 사용할 feature와 타겟 변수 선택
F1 = financial_data[['부채비율','총자산','당기순이익','매출액증가율','자기자본회전율','총자본회전율']]
y1_class = financial_data['레이블']

# 비재무정보에서 사용할 feature와 타겟 변수 선택
F2 = non_financial_data[['최대주주변경','대표이사변경',"전환사채",'파산신청','거래정지','불성실공시법인']]
y2_class = non_financial_data['레이블']


In [83]:
# 데이터 분할 (재무정보만 사용)
X_train_f1, X_test_f1, y_train_f1, y_test_f1 = train_test_split(F1, y1_class, test_size=0.2, random_state=42)

# 데이터 분할 (비재무정보만 사용)
X_train_f2, X_test_f2, y_train_f2, y_test_f2 = train_test_split(F2, y2_class, test_size=0.2, random_state=42)

# 데이터 결합 (재무정보 + 비재무정보)
X_combined = pd.concat([F1, F2], axis=1)
y_combined = y1_class  # Assuming labels are the same for financial and non-financial

X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)


In [66]:
# 모델 학습 및 평가 (재무정보만 사용)
rf_model_f1 = RandomForestClassifier(random_state=42)
rf_model_f1.fit(X_train_f1, y_train_f1)
y_pred_f1 = rf_model_f1.predict(X_test_f1)
print("재무정보만 사용한 모델 성능:")
print(classification_report(y_test_f1, y_pred_f1))

재무정보만 사용한 모델 성능:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92       134
           1       0.88      0.61      0.72        46

    accuracy                           0.88       180
   macro avg       0.88      0.79      0.82       180
weighted avg       0.88      0.88      0.87       180



In [67]:
# 모델 학습 및 평가 (비재무정보만 사용)
rf_model_f2 = RandomForestClassifier(random_state=42)
rf_model_f2.fit(X_train_f2, y_train_f2)
y_pred_f2 = rf_model_f2.predict(X_test_f2)
print("비재무정보만 사용한 모델 성능:")
print(classification_report(y_test_f2, y_pred_f2))

비재무정보만 사용한 모델 성능:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       134
           1       0.87      0.89      0.88        46

    accuracy                           0.94       180
   macro avg       0.92      0.92      0.92       180
weighted avg       0.94      0.94      0.94       180



In [68]:
# 모델 학습 및 평가 (재무정보 + 비재무정보 결합 사용)
rf_model_combined = RandomForestClassifier(random_state=42)
rf_model_combined.fit(X_train_combined, y_train_combined)
y_pred_combined = rf_model_combined.predict(X_test_combined)
print("재무정보와 비재무정보를 결합한 모델 성능:")
print(classification_report(y_test_combined, y_pred_combined))

재무정보와 비재무정보를 결합한 모델 성능:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       134
           1       0.91      0.85      0.88        46

    accuracy                           0.94       180
   macro avg       0.93      0.91      0.92       180
weighted avg       0.94      0.94      0.94       180



In [91]:
# 모델 학습 (전체 특성을 사용하여 중요도 평가)
rf_model_combined = RandomForestClassifier(random_state=42)
rf_model_combined.fit(X_train_combined, y_train_combined)

RandomForestClassifier(random_state=42)

In [92]:
# 특성 중요도 추출
importances = rf_model_combined.feature_importances_
indices = np.argsort(importances)[::-1]

# 상위 5개 특성 선택
top_5_features = X_combined.columns[indices][:5]
print("선정된 상위 5개 특성:", top_5_features)

# 데이터 축소 (선정된 5가지 특성만 사용)
X_train_final = X_train_combined[top_5_features]
X_test_final = X_test_combined[top_5_features]

선정된 상위 5개 특성: Index(['거래정지', '당기순이익', '총자산', '불성실공시법인', '자기자본회전율'], dtype='object')


# 최종 선택한 feature와 이유


최종 선택한 feature:

- 거래정지
- 당기순이익
- 총자산
- 불성실공시법인
- 자기자본회전율

이들 특성은 상장폐지 종목을 예측하는 데 가장 높은 중요도를 가지는 변수들로서, 모델 학습 과정에서 자연스럽게 선택되었습니다.

- 거래정지: 기업이 거래정지 상태에 있다는 것은 상장폐지의 위험이 높다는 신호

- 당기순이익: 기업의 수익성을 나타내는 중요한 지표로, 기업의 재무 건전성을 평가할 때 필수적인 요소

- 총자산: 기업의 자산 규모를 나타내며, 기업의 안정성과 지속 가능성에 중요한 영향

- 불성실공시법인: 공시의무를 성실하게 이행하지 않은 기업은 신뢰성에 문제가 있을 수 있으며, 이는 상장폐지의 원인 제공

- 자기자본회전율: 자기자본을 얼마나 효율적으로 사용하고 있는지를 나타내는 지표로, 기업의 자본 효율성을 평가하는 데 중요도


In [93]:
# 최종 모델 학습 및 평가
final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train_final, y_train_combined)
y_pred_final = final_model.predict(X_test_final)

In [94]:
# 최종 성능 평가
print("선정된 5개 특성만 사용한 모델 최종 성능:")
print(classification_report(y_test_combined, y_pred_final))

선정된 5개 특성만 사용한 모델 최종 성능:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       134
           1       0.91      0.85      0.88        46

    accuracy                           0.94       180
   macro avg       0.93      0.91      0.92       180
weighted avg       0.94      0.94      0.94       180



In [95]:
# f1 score 확인
f1_macro = f1_score(y_test_combined, y_pred_final, average='macro')
print("최종 Macro F1 Score:", f1_macro)

최종 Macro F1 Score: 0.9179070442389816


In [96]:
# 테스트 데이터 로드 및 추가 feature 생성
testDF = pd.read_excel('./data/test.xlsx')

# 테스트 데이터에서 선정된 상위 5개 특성만 사용
test_X = testDF[top_5_features]
test_Y = testDF['레이블']


In [97]:
# 테스트 데이터에 대한 예측
test_pred = final_model.predict(test_X)


In [98]:
# 테스트 데이터 성능 평가
print("테스트 데이터 성능 평가:")
print(classification_report(test_Y, test_pred))

테스트 데이터 성능 평가:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        76
           1       0.92      0.96      0.94        24

    accuracy                           0.97       100
   macro avg       0.95      0.97      0.96       100
weighted avg       0.97      0.97      0.97       100



In [99]:
# f1 score 확인
test_f1_macro = f1_score(test_Y, test_pred, average='macro')
print("테스트 데이터 Macro F1 Score:", test_f1_macro)

테스트 데이터 Macro F1 Score: 0.9594539802676038


# **최종 사용한 모델과 이유**

**최종 사용한 모델:RandomForestClassifier**

RandomForestClassifier는 여러 결정 트리를 결합하여 예측을 수행하는 앙상블 기법으로, 다음과 같은 이유로 선택되었습니다:

- 강력한 예측 성능,
- Feature의 중요도를 평가로 중요한 특성을 식별하고 분석하는 데 유리,
- 과적합 방지
